In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
meta = pd.read_csv(Path.cwd() / 'data' / 'meta.csv', parse_dates=['first_include'], date_format='%Y-%m-%d')

In [3]:
df = pd.read_csv(Path.cwd() / 'data' / 'historical_prices_monthly_stat.csv')
df = df.sort_values(['_code', '_year', '_month'], ascending=True).reset_index(drop=True)

In [4]:
df

,_code,_year,_month,monthly_rtn,monthly_high_end_rtn,monthly_start_high_rtn,daily_rtn_avg,daily_vola,daily_dvola,monthly_mdd
0,AN8068571086,2014,4,0.036648,-0.006457,0.043385,0.001837,0.008701,0.005368,0.019390
1,AN8068571086,2014,5,0.032143,0.000000,0.032143,0.001611,0.007609,0.006297,0.023361
2,AN8068571086,2014,6,0.135117,0.000000,0.135117,0.006489,0.016904,0.006522,0.019861
3,AN8068571086,2014,7,-0.080272,-0.080272,0.000000,-0.003937,0.009125,0.005828,0.079334
4,AN8068571086,2014,8,0.015655,-0.012608,0.028624,0.000832,0.010711,0.007118,0.034285
...,...,...,...,...,...,...,...,...,...,...
201877,US98980L1017,2023,11,0.129935,-0.001766,0.131934,0.006280,0.017968,0.010555,0.037149
201878,US98980L1017,2023,12,0.023047,-0.030993,0.055769,0.001368,0.018804,0.011594,0.043800
201879,US98980L1017,2024,1,-0.065654,-0.083026,0.018944,-0.003228,0.018416,0.012142,0.083026
201880,US98980L1017,2024,2,0.090671,0.000000,0.090671,0.004831,0.023420,0.009059,0.066667


In [5]:
# bins = [-np.inf, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, -0.05, -0.01, 0]
bins = [-np.inf, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, -0.05, -0.01, 0]
labels = [f'({bins[i-1]}, {bins[i]}]' for i, _ in enumerate(bins) if i > 0]
df['monthly_high_end_rtn_category'] = pd.cut(df['monthly_high_end_rtn'], bins=bins, labels=labels).astype(str)
df['before_monthly_high_end_rtn'] = df.groupby('_code', as_index=False)['monthly_high_end_rtn'].shift(1)
df['before_monthly_high_end_rtn_category'] = df.groupby('_code', as_index=False)['monthly_high_end_rtn_category'].shift(1)

In [6]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby('before_monthly_high_end_rtn_category')[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std']).to_excel(Path.cwd() / 'data' / 'test.xlsx')

In [10]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['country', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std']).to_excel(Path.cwd() / 'data' / 'test.xlsx')

In [11]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['gics_sector', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std']).to_excel(Path.cwd() / 'data' / 'test.xlsx')

In [12]:
pd.merge(df, meta[['_code', 'ticker', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry']], how='left', on='_code').groupby(['gics_sector', 'gics_industry_group', 'before_monthly_high_end_rtn_category'])[
    ['monthly_rtn', 'monthly_start_high_rtn', 'daily_rtn_avg', 'monthly_mdd']].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std']).to_excel(Path.cwd() / 'data' / 'test.xlsx')

In [8]:
# import plotly.graph_objects as go

# fig = go.Figure()
# for lb in labels:
#     fig.add_trace(go.Box(x=df[df['before_monthly_high_end_rtn_category'] == lb]['monthly_start_high_rtn'], name=lb, boxpoints=False))
# fig.show()

In [9]:
df['before_monthly_high_end_rtn_category'].value_counts(dropna=False)

before_monthly_high_end_rtn_category
(-0.05, -0.01]    69407
(-0.1, -0.05]     48076
(-0.01, 0]        43905
(-0.2, -0.1]      31039
(-0.3, -0.2]       5303
NaN                2589
(-0.4, -0.3]       1098
(-0.5, -0.4]        262
(-0.6, -0.5]        102
(-inf, -0.7]         55
(-0.7, -0.6]         46
Name: count, dtype: int64